In [33]:
import mlflow
import pkg_resources

from mlflow.types.responses import ResponsesAgentRequest

from agent import LLM_ENDPOINT_NAME, AGENT

In [26]:
input_example = {"input": [{"role": "user", "content": "How many Rs in strawberry?"}]}
# input_example = ResponsesAgentRequest(input=[{"role": "user", "content": "How many Rs in strawberry?"}])

In [35]:
with mlflow.start_run():
    AGENT.predict(input_example)

🏃 View run unique-donkey-737 at: https://adb-3636121482767679.19.azuredatabricks.net/ml/experiments/4353845366834577/runs/c40a629dff0346c6bbf4c9914e719690
🧪 View experiment at: https://adb-3636121482767679.19.azuredatabricks.net/ml/experiments/4353845366834577


In [11]:
resources = [mlflow.models.resources.DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]

In [27]:
mlflow.set_tracking_uri("databricks://test-azure")
mlflow.set_experiment("/Workspace/Users/stuart.lynn@databricks.com/u2m-experiment")

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            f"databricks-langchain=={pkg_resources.get_distribution('databricks-langchain').version}",
            f"langchain-core=={pkg_resources.get_distribution('langchain-core').version}",
            f"databricks-sdk=={pkg_resources.get_distribution('databricks-sdk').version}",
            f"mlflow[databricks]=={pkg_resources.get_distribution('mlflow').version}",
        ]
    )

2025/08/19 12:02:30 INFO mlflow.pyfunc: Predicting on input example to validate output
2025/08/19 12:02:36 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run salty-eel-245 at: https://adb-3636121482767679.19.azuredatabricks.net/ml/experiments/4353845366834577/runs/8201cf97685b45e88b6547e962837267
🧪 View experiment at: https://adb-3636121482767679.19.azuredatabricks.net/ml/experiments/4353845366834577


In [30]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{logged_agent_info.run_id}/agent")
loaded_model.predict(input_example)

{'object': 'response', 'output': []}

In [ ]:
mlflow.models.predict(logged_agent_info.model_uri, input_example, env_manager="uv")

'models:/m-be264c110dd6444584568a79f1561c51'